## Test Time Augmentation

After training our model on the provided images, we would like to generate predictions for the separate test set. Unfortunately, our test images are about 1200x310 and our model can only handle 256x256 inputs.

We can work around this by taking multiple slices of the image, running them through the model and then storing the predictions for each slice.

Some other thoughts:
- How far should we move the 256x256 input when making each set of predictions?
- Is there an intelligent way to average predictions from overlapping slices?
- How do we ensemble predictions from multiple models?

In [1]:
import os
import PIL
import math
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import FloatTensor, LongTensor, tensor

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as patheffects

from RetinaNet.inference import custom_tta, setupNewCrop, get_bounding_box_predictions
from RetinaNet.object_detection_helper import process_output, nms, rescale_boxes, show_results, GeneralEnsemble, tlbr2ltwh

import fastai
from fastai.core import is_tuple
from fastai.torch_core import to_np
from fastai.basic_train import _loss_func2activ, validate, progress_bar, loss_batch
from fastai.vision.data import pil2tensor
from fastai.vision import Path, open_image, ImageBBox, ObjectItemList, get_transforms, bb_pad_collate, conv_layer, Learner, create_body, models, conv2d, ifnone, DatasetType, range_of, progress_bar, ShowGraph, get_preds, Image, load_learner

In [2]:
#from RetinaNet.object_detection_helper import 
from RetinaNet.object_detection_helper import create_anchors, show_results_side_by_side, get_annotations_from_path
from RetinaNet.RetinaNetFocalLoss import FocalLoss
from RetinaNet.RetinaNet import RetinaNet
from RetinaNet.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric

In [3]:
DATA = Path('data/')
IMAGES = DATA/'train_labelled_images'
TEST_IMAGES = DATA/'sorghum_test_public'
ANNOTATIONS = DATA/'train_labelled_annotations'

In [4]:
images, lbl_bbox = get_annotations_from_path(ANNOTATIONS, IMAGES)
img2bbox = dict(zip(images, lbl_bbox))
get_y_func = lambda o:img2bbox[IMAGES/o.name]

In [5]:
def get_data(bs, size):
    src = ObjectItemList.from_folder(IMAGES)
    src = src.split_by_rand_pct(0.1)
    src = src.label_from_func(get_y_func)
    src = src.transform(get_transforms(), size=size, tfm_y=True)
    return src.databunch(path=IMAGES, bs=16, collate_fn=bb_pad_collate)

In [6]:
batch_size = 64
image_size = 256

data = get_data(batch_size, image_size)
data.show_batch(3)

In [7]:
#WARNING: Hardcoded anchor sizes probably not great for when we change things
ratios=[0.5, 1, 2]
scales=[0.35, 0.5, 1]
anchors = create_anchors(sizes=[(32,32),(16,16),(8,8),(4,4),(2,2)], ratios=ratios, scales=scales)

In [8]:
n_classes = data.train_ds.c

crit = FocalLoss(anchors)

encoder = create_body(models.resnet18, True, -2)
model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=9, sizes=[32,16,8,4,2], chs=32, final_bias=-4., n_conv=2)

In [ ]:
voc = PascalVOCMetric(anchors, image_size, [i for i in data.train_ds.y.classes[1:]])
learn = Learner(data, model, loss_func=crit)

In [ ]:
learn.split([model.encoder[6], model.c5top5])
learn.freeze_to(-2)

In [ ]:
learn.fit_one_cycle(20, 2e-3)

epoch,train_loss,valid_loss,time
0,2.830580,1.860166,00:06
1,2.060772,1.564837,00:04
2,1.695979,1.287179,00:04
3,1.454213,1.114911,00:04
4,1.288892,1.228865,00:04
5,1.192719,0.941642,00:04
6,1.085873,0.854033,00:04
7,0.990536,0.774495,00:04
8,0.915224,0.738804,00:04
9,0.859257,0.758050,00:04


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(50, slice(1e-4, 1e-3))

epoch,train_loss,valid_loss,time
0,0.607976,0.616659,00:04
1,0.608998,0.613912,00:04
2,0.615594,0.616569,00:04
3,0.610176,0.609390,00:04
4,0.604139,0.590718,00:04
5,0.603702,0.583682,00:04
6,0.596436,0.584367,00:04
7,0.591628,0.578799,00:04
8,0.585907,0.602350,00:04
9,0.578054,0.574231,00:04


## Get Bounding Boxes via Slicing

One approach to generate our bounding boxes for a full image is to take multiple crops of the image and stick them together. Unfortunately this can be very tricky as we have to keep track of many things:
 - Where are we currently cropping the image?
 - We need to make sure we don't crop too far outside of the image
 - Each image has a unique shape, so we have to keep track of it when cropping

In [ ]:
#NOTE: MOVED TO INFERENCE.py
#NOTE: MOVED TO INFERENCE.py
#NOTE: MOVED TO INFERENCE.py

In [ ]:
#Let's test this out on a single slice and plot the resulting bboxes on to our image
dl = learn.dl(DatasetType.Valid)

old_open_image = fastai.vision.data.open_image

i = 4
j = 1
original_images = list(dl.x)
setupNewCrop(i, j)

all_imgs, all_boxes = get_bounding_box_predictions(learn, dl, anchors, original_images, i,j)

fastai.vision.data.open_image = old_open_image

In [ ]:
img = all_imgs[0]
bboxes = all_boxes[0]

In [ ]:
classId = [0 for _ in range(len(bboxes))]
bbox = ImageBBox.create(*img.size, (bboxes), classId, classes=['d'])
print(img.size)
img.show(figsize=(25,25), y=bbox)

## Ensembling Predictions

Now that we've generated predictions for multiple areas of the image, we would like to combine the predictions somehow. The naieve approach might be to simply concatenate all the bounding boxes into a single list but this won't work due to overlapping windows. Instead we'll have to ensemblem the predictions.

Fortunately this has already been implemented at: https://github.com/ahrnbom/ensemble-objdet

In [ ]:
all_preds = list(custom_tta(learn, anchors))

print(len(all_preds))

In [ ]:
def ensembleBoxesFromSlices(all_preds):
    
    boxes_by_image = {}

    for preds in all_preds:
        images, boxes = preds

        for i, boxes in enumerate(boxes):

            if i not in boxes_by_image:
                boxes_by_image[i] = []

            boxes_by_image[i].append(tlbr2ltwh(boxes))
            
            
    final_preds = []

    for image_index, all_boxes in boxes_by_image.items():
        ensembled_boxes = GeneralEnsemble(all_boxes)

        asdf = []

        for box in ensembled_boxes:
            #[box_x, box_y, box_w, box_h, class, confidence] to [top, left, bottom, right]
            asdf.append([box[1], box[0], box[1] + box[3], box[0] + box[2]])

        final_preds.append(asdf)
        
    return final_preds

In [ ]:
final_preds = ensembleBoxesFromSlices(all_preds)

In [ ]:
classId = [0 for _ in range(len(final_preds[0]))]
bbox = ImageBBox.create(*img.size, final_preds[0], classId, classes=[''])
print(img.size)
img.show(figsize=(15,15), y=bbox)

## Alternate Approach

One alternate approach is to simply pass the full image through our RetinaNet. We can do this because the output volume just represents bounding boxes at different parts of the image, and will always be of the appropriate shape and size.

In [ ]:
learn.data.single_ds.tfmargs['size'] = None

oimg = open_image('/home/josh/git/SorghumHeadDetection/data/train_labelled_images/C11-R39-G345-DSC01757.jpeg')

x = learn.data.one_item(oimg)[0]
with torch.no_grad():
    z = learn.model.eval()(x)

In [ ]:
def process_output(output, i, detect_thresh=0.25):
    "Process `output[i]` and return the predicted bboxes above `detect_thresh`."
    clas_pred,bbox_pred,sizes = output[0][i], output[1][i], output[2]
    anchors = create_anchors(sizes, ratios, scales).to(clas_pred.device)
    bbox_pred = activ_to_bbox(bbox_pred, anchors)
    clas_pred = torch.sigmoid(clas_pred)
    detect_mask = clas_pred.max(1)[0] > detect_thresh
    bbox_pred, clas_pred = bbox_pred[detect_mask], clas_pred[detect_mask]
    bbox_pred = tlbr2cthw(torch.clamp(cthw2tlbr(bbox_pred), min=-1, max=1))    
    scores, preds = clas_pred.max(1)
    return bbox_pred, scores, preds

In [ ]:
from RetinaNet.object_detection_helper import activ_to_bbox, tlbr2cthw, cthw2tlbr, draw_rect

In [ ]:
_,axs = plt.subplots(1, 1, figsize=(15,15))

bbox_pred, scores, preds = process_output(z, 0, detect_thresh=0.5)
to_keep = nms(bbox_pred, scores, thresh=0.1)
bbox_pred, preds, scores = bbox_pred[to_keep].cpu(), preds[to_keep].cpu(), scores[to_keep].cpu()

t_sz = torch.Tensor([*oimg.size])[None].float()
bbox_pred[:,:2] = bbox_pred[:,:2] - bbox_pred[:,2:]/2
bbox_pred[:,:2] = (bbox_pred[:,:2] + 1) * t_sz/2
bbox_pred[:,2:] = bbox_pred[:,2:] * t_sz
bbox_pred = bbox_pred.long()
ax=axs
oimg.show(ax=ax)
for bbox, c, scr in zip(bbox_pred, preds, scores):
    txt = str(c.item()) if learn.data.classes is None else learn.data.classes[c.item()+1]
    draw_rect(ax, [bbox[1],bbox[0],bbox[3],bbox[2]], text=f'{scr:.2f}')

Unfortunately, these bounding boxes don't seem to be as accurate as the more complicated approach. :(